<a href="https://colab.research.google.com/github/i2mmmmm/kaggle_flood/blob/main/4.%20ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
os.environ['KAGGLE_USERNAME']='mmmmmlee'
os.environ['KAGGLE_KEY']='b4527f98e27813c6dae3fb199096dcb5'

In [2]:
!kaggle competitions download -c Playground-series-s4e5

 61% 17.0M/28.0M [00:00<00:00, 64.4MB/s]
100% 28.0M/28.0M [00:00<00:00, 92.5MB/s]


In [3]:
!unzip 'Playground-series-s4e5.zip'

Archive:  Playground-series-s4e5.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [27]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.ensemble import VotingRegressor, BaggingRegressor, StackingRegressor
from sklearn.linear_model import BayesianRidge, SGDRegressor, Ridge
from sklearn.model_selection import cross_val_score, KFold
from sklearn.pipeline import make_pipeline
import xgboost as xgb
import lightgbm as lgb

from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping

In [7]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [54]:
# train 표준화, 스케일링
scaler = StandardScaler()
X = train.drop(columns=['FloodProbability', 'id'])
y = train['FloodProbability']
X_scaled = scaler.fit_transform(X)

In [55]:
# 선형회귀_baseline

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=35)
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print('R2 Score:', r2_score(y_test, y_pred))
print('MSE:', mean_squared_error(y_test, y_pred))

R2 Score: 0.8449835489068076
MSE: 0.00040395088979260915


In [22]:
# 다시 feature engineering
train = pd.read_csv('train.csv')

def simplified_getFeats(df):
    num_cols = ['MonsoonIntensity', 'TopographyDrainage', 'RiverManagement', 'Deforestation', 'Urbanization', 'ClimateChange', 'DamsQuality', 'Siltation', 'AgriculturalPractices', 'Encroachments', 'IneffectiveDisasterPreparedness', 'DrainageSystems', 'CoastalVulnerability', 'Landslides', 'Watersheds', 'DeterioratingInfrastructure', 'PopulationScore', 'WetlandLoss', 'InadequatePlanning', 'PoliticalFactors']

    scaler = StandardScaler().fit(df[num_cols])
    df[num_cols] = scaler.transform(df[num_cols])  # Scale early

    # Basic Stats
    df['mean'] = df[num_cols].mean(axis=1)
    df['std'] = df[num_cols].std(axis=1)
    df['max'] = df[num_cols].max(axis=1)
    df['min'] = df[num_cols].min(axis=1)

    # Interaction Features (Simplified)
    df['Climate_Risk'] = df['MonsoonIntensity'] * df['ClimateChange']
    df['Infrastructure_Risk'] = df['DamsQuality'] * df['DrainageSystems']
    df['wet_Risk'] = df['WetlandLoss'] * df['Encroachments']

    return df

train_simp = simplified_getFeats(train)

X = train_simp.drop(columns=['FloodProbability', 'id'])
y = train_simp['FloodProbability']

In [28]:
# 훈련, 검증, 테스트 데이터 나누기
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=35)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=35)

# 모델 설계
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(32, activation='relu'))
model.add(Dense(1))  # 출력 뉴런 하나 (홍수 발생 가능성)

# 모델 컴파일
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# EarlyStopping 콜백 설정
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# 모델 훈련
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val))

# 모델 평가
loss, mae = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test MAE:", mae)

Epoch 1/50
24456/24456 [==============================] - 70s 3ms/step - loss: 9.5578e-04 - mae: 0.0188 - val_loss: 4.4206e-04 - val_mae: 0.0173
Epoch 2/50
24456/24456 [==============================] - 56s 2ms/step - loss: 3.7532e-04 - mae: 0.0153 - val_loss: 3.6232e-04 - val_mae: 0.0150
Epoch 3/50
24456/24456 [==============================] - 58s 2ms/step - loss: 3.7033e-04 - mae: 0.0152 - val_loss: 3.6018e-04 - val_mae: 0.0150
Epoch 4/50
24456/24456 [==============================] - 59s 2ms/step - loss: 3.6829e-04 - mae: 0.0151 - val_loss: 3.6000e-04 - val_mae: 0.0151
Epoch 5/50
24456/24456 [==============================] - 59s 2ms/step - loss: 3.6758e-04 - mae: 0.0151 - val_loss: 3.6595e-04 - val_mae: 0.0153
Epoch 6/50
24456/24456 [==============================] - 56s 2ms/step - loss: 3.6771e-04 - mae: 0.0151 - val_loss: 3.6651e-04 - val_mae: 0.0153
Epoch 7/50
24456/24456 [==============================] - 57s 2ms/step - loss: 3.6670e-04 - mae: 0.0151 - val_loss: 3.6466e-04 - v

In [32]:
# test에도 적용
test = pd.read_csv('test.csv')
test_simp = simplified_getFeats(test)


In [33]:
# test_simp 데이터셋에서 id를 제외한 특성만 사용하여 예측
X_test_simp = test_simp.drop(columns=['id'])
y_pred_simp = model.predict(X_test_simp)

# 예측 결과를 DataFrame으로 저장
predictions = pd.DataFrame({'id': test_simp['id'], 'PredictedFloodProbability': y_pred_simp.flatten()})

# 예측 결과 출력
print(predictions.head())

23291/23291 [==============================] - 34s 1ms/step
        id  PredictedFloodProbability
0  1117957                   0.577113
1  1117958                   0.455221
2  1117959                   0.450350
3  1117960                   0.469356
4  1117961                   0.469919


In [34]:
predictions.to_csv('submission_MLP.csv', index=False)

In [24]:
# test flood 예측 진행

X_train = train_simp.drop(columns=['FloodProbability', 'id'])
y_train = train_simp['FloodProbability']
X_test = test_simp.drop(columns=['id'])

In [ ]:
# 모델 설계
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(32, activation='relu'))
model.add(Dense(1))  # 출력 뉴런 하나 (홍수 발생 가능성)

# 모델 컴파일
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# EarlyStopping 콜백 설정
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# 모델 훈련
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val))

In [15]:
# LightGBM 모델 학습

lgbm_params = {
    'num_leaves': 183,
    'learning_rate': 0.01183688880802108,
    'n_estimators': 577,
    'subsample_for_bin': 165697,
    'min_child_samples': 114,
    'reg_alpha': 2.075080888948164e-06,
    'reg_lambda': 3.838938366471552e-07,
    'colsample_bytree': 0.9634044234652241,
    'subsample': 0.9592138618622019,
    'max_depth': 9
}

model_lgb = lgb.LGBMRegressor(**lgbm_params)
model_lgb.fit(X_train, y_train)
y_pred_lgb = model_lgb.predict(X_test)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.487907 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1843
[LightGBM] [Info] Number of data points in the train set: 1117957, number of used features: 27
[LightGBM] [Info] Start training from score 0.504480
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gai

In [16]:
# 출력
test_simp['FloodProbability'] = y_pred_lgb
test_simp[['id', 'FloodProbability']].to_csv('submission_engi_lgb_para.csv', index=False)

# 특성 엔지니어링 이후 제출 - 0.866

In [44]:
sub1 = pd.read_csv('submission (1).csv')
sub2 = pd.read_csv('submission (2).csv')
sub3 = pd.read_csv('submission (3).csv')

print(sub1.head())
print(sub2.head())
print(sub3.head())

        id  FloodProbability
0  1117957          0.578705
1  1117958          0.454723
2  1117959          0.448682
3  1117960          0.467035
4  1117961          0.465906
        id  FloodProbability
0  1117957          0.577933
1  1117958          0.456926
2  1117959          0.447441
3  1117960          0.464352
4  1117961          0.466865
        id  FloodProbability
0  1117957          0.578782
1  1117958          0.454955
2  1117959          0.448574
3  1117960          0.466518
4  1117961          0.466017


In [59]:
sub5 = pd.read_csv('submission_MLP.csv')

In [69]:
sub1

,id,FloodProbability,a,b,c,d,ensemble,ensemble2
0,1117957,0.578705,0.577933,0.578782,0.577174,0.577113,0.578473,0.578937
1,1117958,0.454723,0.456926,0.454955,0.454428,0.455221,0.455535,0.453912
2,1117959,0.448682,0.447441,0.448574,0.450117,0.450350,0.448232,0.449132
3,1117960,0.467035,0.464352,0.466518,0.466850,0.469356,0.465968,0.468103
4,1117961,0.465906,0.466865,0.466017,0.467364,0.469919,0.466263,0.465550
...,...,...,...,...,...,...,...,...
745300,1863257,0.476473,0.475038,0.475655,0.477278,0.475582,0.475722,0.477223
745301,1863258,0.441803,0.442251,0.442204,0.443863,0.446894,0.442086,0.441521
745302,1863259,0.622910,0.622623,0.621720,0.622035,0.624680,0.622418,0.623402
745303,1863260,0.548660,0.548482,0.548708,0.549549,0.553482,0.548617,0.548703


In [68]:
sub1['ensemble2']=ensemble2['FloodProbability']

In [45]:
print(len(sub1))
print(len(sub2))
print(len(sub3))

745305
745305
745305


In [50]:
# 예측값 평균 내기
ensemble_sub = sub1.copy()
ensemble_sub['FloodProbability'] = (sub1['FloodProbability'] + sub2['FloodProbability'] + sub3['FloodProbability']) / 3

# 앙상블 결과 저장
ensemble_sub.to_csv('submission_ensemble.csv', index=False)

In [66]:
ensemble2 = sub2.copy()
ensemble2['FloodProbability'] = sub1['FloodProbability']  +sub1['FloodProbability'] - ensemble_sub['FloodProbability']

In [70]:
ensemble2.to_csv('submission_ensemble2.csv', index=False)

In [71]:
len(ensemble2)

745305

In [52]:
print(ensemble2.head())

        id  FloodProbability
0  1117957          0.578473
1  1117958          0.455535
2  1117959          0.448232
3  1117960          0.465968
4  1117961          0.466263


In [51]:
len(ensemble_sub)

745305